<a href="https://colab.research.google.com/github/ManasviAtGitHub/Algorithms-for-Optimization/blob/main/Search_Methods_(Fibonacci%2C_Golden_Section%2C_Quadratic_Fit).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <img src="https://github.com/JuliaLang/julia-logo-graphics/raw/master/images/julia-logo-color.png" height="100" /> _Colab Notebook Template_

## Instructions
1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. If you need a GPU: _Runtime_ > _Change runtime type_ > _Harware accelerator_ = _GPU_.
3. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia and other packages (if needed, update `JULIA_VERSION` and the other parameters). This takes a couple of minutes.
4. Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the next section.

_Notes_:
* If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2, 3 and 4.
* After installation, if you want to change the Julia version or activate/deactivate the GPU, you will need to reset the Runtime: _Runtime_ > _Factory reset runtime_ and repeat steps 3 and 4.

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.6.0" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools Plots"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -n "$COLAB_GPU" ] && [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  if [ "$COLAB_GPU" = "1" ]; then
      JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Success! Please reload this page and jump to the next section."
fi

Installing Julia 1.6.0 on the current Colab Runtime...
2021-10-30 05:01:50 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.6/julia-1.6.0-linux-x86_64.tar.gz [112838927/112838927] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
Installing Julia package BenchmarkTools...
Installing Julia package Plots...
Installing IJulia kernel...
[ Info: Installing julia kernelspec in /root/.local/share/jupyter/kernels/julia-1.6

Success! Please reload this page and jump to the next section.


# Need Help?

* Learning: https://julialang.org/learning/
* Documentation: https://docs.julialang.org/
* Questions & Discussions:
  * https://discourse.julialang.org/
  * http://julialang.slack.com/
  * https://stackoverflow.com/questions/tagged/julia

If you ever ask for help or file an issue about Julia, you should generally provide the output of `versioninfo()`.

Add new code cells by clicking the `+ Code` button (or _Insert_ > _Code cell_).

Have fun!

<img src="https://raw.githubusercontent.com/JuliaLang/julia-logo-graphics/master/images/julia-logo-mask.png" height="100" />

## Functions for which we will try to find minimum using the search methods

In [1]:
g=x->x^2

#1 (generic function with 1 method)

In [2]:
g(5)

25

In [3]:
h=x->((x^2)+(2*x)+5)

#3 (generic function with 1 method)

In [4]:
h(-1)

4

In [24]:
d=x->(4*(x^2)+(10*x)+10)

#9 (generic function with 1 method)

## Funtion to find intervals ie brackets in which global minimum must lie

In [5]:
function bracket_minimum(fun1,x=0, s=1e-2, k=2.0)
    a, ya = x, fun1(x)
    b, yb = a + s, fun1(a + s)
    if yb > ya
        a, b = b, a
        ya, yb = yb, ya
        s = -s
    end
    while true
        c, yc = b + s, fun1(b + s)
        if yc > yb
            return a < c ? (a, c) : (c, a)
        end
        a, ya, b, yb = b, yb, c, yc
        s *= k
    end
end

bracket_minimum (generic function with 4 methods)

In [6]:
bracket_minimum(g)

(-0.01, 0.01)

In [7]:
bracket_minimum(h)

(-2.55, -0.6300000000000001)

Fibonacci Search Method

In [8]:
s=(1-√5)/(1+√5)

-0.38196601125010515

In [9]:
φ = MathConstants.φ

φ = 1.6180339887498...

In [10]:
phi = 1.618

1.618

Fn-1/Fn

In [11]:
smallerToLargerRatio=n-> 1 / (φ*(1-s^(n+1))/(1-s^n))

#5 (generic function with 1 method)

Fn/Fn-1

In [12]:
largerToSmallerRatio=n-> φ*(1-s^(n+1))/(1-s^n)

#7 (generic function with 1 method)

In [13]:
smallerToLargerRatio(100)

0.6180339887498948

In [14]:
largerToSmallerRatio(100)

1.618033988749895

### Rules that must be satisfied for applying such search methods
[a,b]

a<b<c

f(a)>f(b)<f(c) 

## Fibonacci Search

In [15]:
function fibonacci_search(g, a, b, n, ϵ=0.01)
    s = (1-√5)/(1+√5)
    ρ = 1 / (φ*(1-s^(n+1))/(1-s^n))
    d = ρ*b + (1-ρ)*a
    yd = g(d)
    for i in 1 : n-1
        if i == n-1
            c = ϵ*a + (1-ϵ)*d
        else
            c = ρ*a + (1-ρ)*b
        end
        yc = g(c)
        if yc < yd
            b, d, yd = d, c, yc
        else
            a, b = b, c
        end
        ρ = 1 / (φ*(1-s^(n-i+1))/(1-s^(n-i)))
    end
    return a < b ? (a, b) : (b, a)
end

fibonacci_search (generic function with 2 methods)

In [16]:
fibonacci_search(g,-1,1,10)

(-0.011235955056179792, 0.011235955056179796)

In [17]:
fibonacci_search(h,-1.5,0.5,25)

(-1.0000041188536406, -0.999987643439078)

### Debugging the search method

In [18]:
function fibonacci_search_rough(g, a, b, n, ϵ=0.01)
    s = (1-√5)/(1+√5)
    ρ = 1 / (φ*(1-s^(n+1))/(1-s^n))
    d = ρ*b + (1-ρ)*a
    println("d $d")
    yd = g(d)
    println("yd $yd")
    for i in 1 : n-1
        if i == n-1
            c = ϵ*a + (1-ϵ)*d
            println("eps_c $c")
        else
            c = ρ*a + (1-ρ)*b
            println("c $c")
        end
        yc = g(c)

        println("yc $yc")
        if yc < yd
            b, d, yd = d, c, yc
        else
            a, b = b, c
        end
        
        ρ = 1 / (φ*(1-s^(n-i+1))/(1-s^(n-i)))
    end
    return a < b ? (a, b) : (b, a)
end

fibonacci_search_rough (generic function with 2 methods)

In [19]:
fibonacci_search_rough(g,-1,1,5)

d 0.24999999999999978
yd 0.06249999999999989
c -0.24999999999999978
yc 0.06249999999999989
c 0.5000000000000002
yc 0.2500000000000002
c 2.498001805406602e-16
yc 6.240013019814644e-32
eps_c -0.0024999999999997507
yc 6.2499999999987535e-6


(-0.0024999999999997507, 0.24999999999999978)

## Golden Section Search

In [20]:
function golden_section_search(f, a, b, n)
    ρ = φ-1
    d = ρ * b + (1 - ρ)*a
    fd = f(d)
    for i = 1 : n-1
        c = ρ*a + (1 - ρ)*b
        fc = f(c)
        if fc < fd
            b, d, fd = d, c, fc
        else
            a, b, fb = b, c, fc
        end
    end
    return a < b ? (a, b) : (b, a)
end

golden_section_search (generic function with 1 method)

In [21]:
golden_section_search(g,-1,1,10)

(-0.01315561749642492, 0.013155617496424812)

In [22]:
golden_section_search(h,-1.5,0.5,10)

(-1.011602807488854, -0.9852915724960042)

## Quadratic Fit Search

In [25]:
function quadratic_fit_search(f, a, b, c, n)
    ya, yb, yc = f(a), f(b), f(c)
    for i in 1:n-3
        x = 0.5*(ya*(b^2-c^2)+yb*(c^2-a^2)+yc*(a^2-b^2)) /(ya*(b-c) +yb*(c-a) +yc*(a-b))
        yx = f(x)
        if x > b
            if yx > yb
                c, yc = x, yx
            else
                a, ya, b, yb = b, yb, x, yx
            end
        elseif x < b
            if yx > yb
                a, ya = x, yx
            else
                c, yc, b, yb = b, yb, x, yx
            end
        end
    end
    return (a, b, c)
end

quadratic_fit_search (generic function with 1 method)

In [26]:
quadratic_fit_search(h,-3,-2,2,30)

(-2, -1.0, 2)

In [27]:
quadratic_fit_search(g,-3,-2,1,30)

(-2, -0.0, 1)

In [28]:
quadratic_fit_search(d,-2.5,-2,-1,30)

(-2, -1.25, -1)